# Interacting with impala using python

Connecting to impala is easy using the [impyla](https://github.com/cloudera/impyla) or [ibis](https://github.com/cloudera/ibis) module. Since the Ibis module is still under devlopment, this tutorial will only cover impyla.

## Impyla Module

The Impyla module is easy to use if you know SQL. You can simply write a query, execute it in Python, and pull the results down in a pandas dataframe for downstrem analysis. 

### Run First Time: Install impyla

In [ ]:
pip install impyla

If pip is not installed on your system, you can visit [this website](http://pip.readthedocs.org/en/stable/installing/) or install pip using anaconda:

In [ ]:
conda install pip

### Creating a Connection

To connect to impala, first create a connection string specifying your impala hostname and port. The default port for impala is 21050.

In [6]:
from impala.dbapi import connect

#create a connection, replace 'impala_host' with your host name
conn=connect(host=my_host, port=21050)

Once you setup the connection string, you can create a cursor object for intereacting with the database:

In [2]:
#create a cursor object to interact with the db
cur = conn.cursor()

In [3]:
# view cursor object
print cur

In [5]:
test = "show databases"
cur.execute(test)

#print test_results# print results
for row in cur.fetchall():
    print row

('_impala_builtins',)
('clarity2',)
('default',)
('p7_archive',)
('p7_itmi',)
('p7_platform',)
('p7_product',)
('p7_ref_grch37',)
('p7_ref_hg19',)
('p7_staging',)
('training',)


### Run Queries

Python interacts with impala by executing sql queries using cur.execute() to execute the query, and then using cur.fetchall() to grab the results.

### Print results to screen

In [16]:
test_query = 'SELECT * from p7_ref_grch37.cytoband limit 5'

# execute sql query
cur.execute(test_query)

#print test_results# print results
for row in cur.fetchall():
    print row

('1', 0, 2300000, 'p36.33', 'gneg')
('1', 2300000, 5400000, 'p36.32', 'gpos25')
('1', 5400000, 7200000, 'p36.31', 'gneg')
('1', 7200000, 9200000, 'p36.23', 'gpos25')
('1', 9200000, 12700000, 'p36.22', 'gneg')


### Save results as pandas table

In [18]:
# import pandas impala api
from impala.util import as_pandas 

# execute sql query
cur.execute(test_query)

# save results as dataframe
results = as_pandas(cur)

print results

  chrom    start      stop    name gie_stain
0     1        0   2300000  p36.33      gneg
1     1  2300000   5400000  p36.32    gpos25
2     1  5400000   7200000  p36.31      gneg
3     1  7200000   9200000  p36.23    gpos25
4     1  9200000  12700000  p36.22      gneg


### View available databases and tables

To see what databases are available in impala, let's run a simple SQL statement to view available databases 'SHOW databases'. 

In [19]:
#view available databases
cur.execute('SHOW DATABASES')

#fetch results of cur.execute()
for row in cur.fetchall():
    print row

('__ibis_tmp',)
('_impala_builtins',)
('cagi_pgp_2015',)
('clarity2',)
('default',)
('p7_archive',)
('p7_itmi',)
('p7_platform',)
('p7_product',)
('p7_ref_grch37',)
('p7_staging',)
('training',)
('users_hrishi',)
('users_selasady',)
('users_vdhankan',)


Let's take a look at the public resources available for the grch37 build by first selecting that database (p7_ref_grch37) with a SQL USE statement 'USE p7_ref_grch37', then asking to see all the tables in that database:

In [20]:
#select a particular database to use
cur.execute('USE p7_ref_grch37')

#view tables in selected database 
#if no db is selected, you will see tables in default db
cur.execute('SHOW TABLES')

# view results
for row in cur.fetchall():
    print row

('acmg_ensembl',)
('acmg_genes',)
('cadd',)
('cadd_tsv',)
('clinvar',)
('clinvar_summary',)
('cytoband',)
('dann',)
('dbnsfp_variant',)
('dbsnp',)
('ensembl_genes',)
('esp_exomes',)
('go_goa',)
('kaviar',)
('kaviar_isb',)
('mirbase',)
('nbs_ensembl',)
('nbs_genes',)
('pfam_clans',)
('pfam_regions',)
('refseq',)
('refseq_map',)
('repeatmasker',)
('ucsc_genes',)
('ucsc_genomicsuperdups',)
('ucsc_kgxref',)
('ucsc_knowngene',)
('uniprot',)


### Viewing table information

In order to match up fields from different tables, it helps to have more information about what each table contains. The SQL "DESCRIBE" statement can be used to find out column names, data types and a description of the contents:

In [21]:
cur.execute('DESCRIBE p7_ref_grch37.cytoband')
for row in cur.fetchall():
    print row

('chrom', 'string', 'Chromosome number')
('start', 'int', 'Start position in genoSeq (renamed from chromstart)')
('stop', 'int', 'End position in genoSeq (renamed from chromend)')
('name', 'string', 'Name of cytogenetic band')
('gie_stain', 'string', 'Giesma stain results (renamed from gieStain)')


## Connect with Ibis

Ibis provides a lot more functionality than Impyla and instead of writing SQL queries, you can use a syntax almost identical to the syntax in the Pandas module, making it very easy to work with tabular data. 

In Ibis, you have to explicity let python know when to pull results into memory, making it much quicker to subset and filter data than with the impyla module. 

### Create connection object

In [1]:
import ibis
import os

# connect to impala with ibis
hdfs_port = os.environ.get('glados16', 50090)
hdfs = ibis.hdfs_connect(host='glados16', port=hdfs_port, user='hdfs')
con = ibis.impala.connect(host='glados19', port=21050, timeout=120)

# enable interactive mode
ibis.options.interactive = True

### View Databases

In [2]:
con.list_databases()

['__ibis_tmp',
 '_impala_builtins',
 'cagi_pgp_2015',
 'clarity2',
 'default',
 'p7_archive',
 'p7_itmi',
 'p7_platform',
 'p7_product',
 'p7_ref_grch37',
 'p7_staging',
 'training',
 'users_hrishi',
 'users_selasady',
 'users_vdhankan']

### List tables

In [29]:
con.list_tables(database='p7_ref_grch37')

['acmg_ensembl',
 'acmg_genes',
 'cadd',
 'cadd_tsv',
 'clinvar',
 'clinvar_summary',
 'cytoband',
 'dann',
 'dbnsfp_variant',
 'dbsnp',
 'ensembl_genes',
 'esp_exomes',
 'go_goa',
 'kaviar',
 'kaviar_isb',
 'mirbase',
 'nbs_ensembl',
 'nbs_genes',
 'pfam_clans',
 'pfam_regions',
 'refseq',
 'refseq_map',
 'repeatmasker',
 'ucsc_genes',
 'ucsc_genomicsuperdups',
 'ucsc_kgxref',
 'ucsc_knowngene',
 'uniprot']

### Connect to a table

In [35]:
db = 'p7_ref_grch37'

# connect to variant tables
cytoband = con.table('cytoband', database=db)

print cytoband.limit(5)

  chrom    start      stop    name gie_stain
0     1        0   2300000  p36.33      gneg
1     1  2300000   5400000  p36.32    gpos25
2     1  5400000   7200000  p36.31      gneg
3     1  7200000   9200000  p36.23    gpos25
4     1  9200000  12700000  p36.22      gneg


### Downloading table object as pandas dataframe

In [36]:
cytoband_df = con.table('cytoband', database=db).execute()
print cytoband_df.head(5)

  chrom    start      stop    name gie_stain
0     1        0   2300000  p36.33      gneg
1     1  2300000   5400000  p36.32    gpos25
2     1  5400000   7200000  p36.31      gneg
3     1  7200000   9200000  p36.23    gpos25
4     1  9200000  12700000  p36.22      gneg


### Save table to impala

To create a new table on impala, you first need to specify the table schema in ('colname', 'data_type') format as follows. 

In the next release of Ibis, you will be able to save a pandas dataframe directly to Ibis. 

In [42]:
# create table schema
test_schema = ibis.schema([
    ('chrom', 'string'), 
    ('start', 'int32'),
    ('stop', 'int32'),
    ('name', 'string'),
    ('gie_stain', 'string')
        ])

# upload table to impala
con.create_table('cytoband_test', cytoband.limit(5), schema=test_schema, database='training')

## Close connection to impala

Once you are finished with a query, it's important to close the connection object. 

In [43]:
# impyla
conn.close()

#ibis
con.close()